## What is Linear Programming?



In [1]:
from IPython.display import IFrame
IFrame("https://www.sciencedirect.com/topics/earth-and-planetary-sciences/linear-programming", width = 1200, height = 400)


For Exapmple:

Maximise: $5X_{1} + 3X_{2}$ </br>
Subject to:
> $3X_{1} + X_{2} <= 3$ </br>
> $X_{1} - X_{2} <= 5$

***

<b>Terminology</b>

 1. <b>Objective Function: </b> The function that is to be maximised/minimised
 2. <b>Decision Variables: </b> Controllable Inputs
 3. <b>Parameters: </b> Uncontrollable inputs or constant numerical values
 4. <b>Constraints: </b> What requirements must be met for optimising the objective function

<b>How is an LP problem solved?</b>

- Graphical Methods
- Simplex Methods
- Dual Simplex Methods
- Branch and Cut (For Integer Linear Programming) etc.

From coding perspective, there are many solvers available which can automatically solve these equations, using the above methods under the hood (like the CPLEX solver by IBM uses the simplex method for solving LP problems)
</br>
The main task is to formulate the business problem as an LP problem!

$z = c_{1}x_{1} + c_{2}x_{2} + ... + c_{n}x_{n}$

***

<font color=red size=5>Formulating an LP problem is hard!</font>

For e.g:

Problem: Max $X$
</br>
Subject to: $X < 2$
</br>

This very simple problem has no solution

<b>Another Example</b>

Problem: Maximize $X_{2}$

Subject to: 
> $X_{1} + X_{2} <= 2$
</br>
> $X_{1}^{2} <= 4$

This problem may seem like it's not an LP problem due to the non-linear constraint, but <b>it is actually an LP problem</b>!

<b>How? </b>
</br>
Although it looks like a non-linear constraint, it can be equivalently written down as:
</br>
> $X_{1} >= -2, 
 X_{2} <= 2$
 
These indeed are linear constraints which can be solved using LP

***

#### Problem Statement

Minimise the number of trucks required to carry a set of boxes by optimising the loading of the boxes onto the trucks.

</br>

</br>

<b>What does "optimal loading" refer to? </b>

- We load the boxes onto the trucks subject to the following constraints:
   - The weight of the boxes loaded should not exceed the permissible  weight for the truck.
   -  The volume of the boxes loaded should not exceed the total volume of the truck.
   -  Every position in the truck will have a minimum permissible weight till which boxes can be stacked on top of each other on that position.
   -  Only similar or lesser volume boxes are allowed to be stacked on a particular box
   -  Rotational Constraints. Some boxes cannot be rotated along the axes, some boxes can only be rotated along the horizontal axis  <font color=red>(not addressed!)</font>
   - Centre of Gravity related constraints etc.  <font color=red>(not addressed!)</font>

***

### Importing the Libaries

In [2]:
import numpy as np
import pandas as pd
from pulp import *
import math
import itertools
import pymorton

### Sample Data and Truck Dimesions

In [3]:
#Boxes Data

data = pd.DataFrame()
data['boxes'] = [0, 1]
data['weight'] =[150, 150]


data['length'] = [200, 200]
data['breadth'] = [150, 150]
data['height'] = [60, 55]
data['volume'] = list(np.array(data['length']) * np.array(data['breadth']) * np.array(data['height']))
#     df['truck_weight'] = 1000
data.head()

,boxes,weight,length,breadth,height,volume
0,0,150,200,150,60,1800000
1,1,150,200,150,55,1650000


In [4]:
#Dimensions of truck, axes

max_trucks = [i for i in range(data.shape[0])]
length_of_truck = 200
breadth_of_truck = 150
height_of_truck = 120

min_height = min(data['height'].values)
min_breadth = min(data['breadth'].values)
min_length = min(data['length'].values)
volume_of_truck = length_of_truck * breadth_of_truck * height_of_truck
weight_of_truck = 800
x_axis = math.ceil(length_of_truck / min_length)
y_axis = math.ceil(breadth_of_truck / min_breadth)
z_axis = math.ceil(height_of_truck / min_height)

max_weight_for_each_stack = 350

### Defining the Decision Variables

Two decision variables

 - $Y_{j}$ --> 1 if truck j is used, else 0
 - $X_{i, j, k}$ --> 1 if $box_{i}$ can be placed in $truck_{j}$ at position k, else 0

1. For defining $Y_{j}$, we first consider it to be a maximum value (= number of boxes considering only one box fits in each truck), then minimise it as per our constraints!
</br>
2. For defining the position k inside a truck:
    - Every position inside a truck can be expressed as a 3-dimensional (x, y, z) coordinate.
    - We can define the decision variable like $X_{i,j,x,y,z}$ but this is inefficient and defining and solving the constraints will be much slower!
    - So, we encode the (x,y,z) coordinates as a single k using <b>Morton Codes</b>
    
</br>

### Morton Codes


In [5]:

IFrame("https://en.wikipedia.org/wiki/Z-order_curve", width = 1200, height = 350)

e.g:

In [6]:
pymorton.interleave(1,2,3)

53

In [7]:
#Decision Variable -> y (if truck i is used or not)
y = [LpVariable("t{0}".format(i+1), cat=LpBinary) for i in max_trucks]

#Decision Variablee -> boxes_loaded (1 if box i is loaded in truck j at position k)
list_of_axes = [[x for x in range(x_axis)] , [y for y in range(y_axis)] , [z for z in range(z_axis)]]
list_coordinates = list(itertools.product(*list_of_axes))
box_loaded = LpVariable.dicts("At_truck_location",
                                   [(i,j, k) for i in data['boxes']
                                    for j in max_trucks
                                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]],
                                   0, 1, LpBinary)

### Define the Problem

In [8]:
problem = LpProblem("truck_loading", LpMinimize)

### Define the Objective 

In [9]:
problem += lpSum([y[i] for i in max_trucks])

In [10]:
#problem

##### Adding Constraints

In [11]:
#For each box, it can be placed in only one truck

for i in data['boxes']:
    problem += lpSum(box_loaded[(i, j, k)] for j in max_trucks
                 for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) == 1

In [12]:
#for each truck,
#    summation of weight of all boxes should be less than total weight truck can carry
#    summation of volume of all boxes should be less than volume of the truck


for j in max_trucks:
    problem += lpSum(box_loaded[(i, j, k)] * data['weight'][i] for i in data['boxes'] 
                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= weight_of_truck * y[j]
    problem += lpSum(box_loaded[(i, j, k)] * data['volume'][i] for i in data['boxes'] 
                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= volume_of_truck * y[j]


In [13]:
# for each truck,
#     for each x,y point
#         1. sum of all boxes placed along the z axis on that point should be less than height of truck
#         2. weight of all boxes along a stack must be less than permissible weight


for j in max_trucks:
    for y_ in range(y_axis):
        for x_ in range(x_axis):
            problem += lpSum(box_loaded[(i, j, k)] * data['height'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for z_ in range(z_axis)]) <= height_of_truck * y[j]
            
            problem += lpSum(box_loaded[(i, j, k)] * data['weight'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for z_ in range(z_axis)]) <= max_weight_for_each_stack * y[j]
            

In [14]:
# for each truck,
#     for each x,z point
#         sum of all boxes placed along the y axis on that point should be less than width of truck


for j in max_trucks:
    for z_ in range(z_axis):
        for x_ in range(x_axis):
#             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
            problem += lpSum(box_loaded[(i, j, k)] * data['breadth'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for y_ in range(y_axis)]) <= breadth_of_truck * y[j]

In [15]:
# for each truck,
#     for each y,z point
#         sum of all boxes placed along the x axis on that point should be less than length of truck


for j in max_trucks:
    for z_ in range(z_axis):
        for y_ in range(y_axis):
#             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
            problem += lpSum(box_loaded[(i, j, k)] * data['length'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for x_ in range(x_axis)]) <= length_of_truck * y[j]

In [16]:
# for j in max_trucks:
#     for y_ in range(y_axis):
#         for x_ in range(x_axis):
#             for z_ in range(1, z_axis):
#                 for i in range(data['boxes'].shape[0]):
#                     k = pymorton.interleave(x_, y_, z_)
#                     k_ = pymorton.interleave(x_, y_, z_-1)
#                     problem += (box_loaded[(i, j, k_)] * data['length'].iloc[i] * data['breadth'].iloc[i])-(box_loaded[(i, j, k)] * data['length'].iloc[i] * data['breadth'].iloc[i]) >= 999999*(1-y[j]) 

In [17]:
#problem

In [18]:
problem.solve()

1

In [19]:
s

NameError: name 's' is not defined

In [ ]:
TOL = 0.00001
truck_num = 0
total_trucks_used = 0
for j in max_trucks:
    if y[j].varValue > TOL:
        total_trucks_used += 1
print(f"Total Trucks Used: {total_trucks_used}")

In [ ]:
for j in max_trucks:
    if y[j].varValue > TOL:
        print(f"Truck_{j} Loads:")
    for i in data['boxes']:
        
        for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]:
            
            if box_loaded[(i, j, k)].varValue:
                
                print(f"box_{i}_loaded_in_truck_{j}")

In [ ]:
# problem

In [ ]:
class OptimiseBoxLoading:
    def __init__(self):
        pass
    def define_parameters(self, df_truck, df_boxes):
        length_of_truck = df_truck['truck_length'].iloc[0]
        breadth_of_truck = df_truck['truck_width'].iloc[0]
        height_of_truck = df_truck['truck_height'].iloc[0]
        volume_of_truck = df_truck['truck_capacity'].iloc[0]
        weight_of_truck = df_truck['truck_weight_limit'].iloc[0]

        min_height = df_boxes['height'].min()
        min_breadth = df_boxes['breadth'].min()
        min_length = df_boxes['length'].min()
        
        x_axis = math.ceil(length_of_truck / min_length)
        y_axis = math.ceil(breadth_of_truck / min_breadth)
        z_axis = math.ceil(height_of_truck / min_height)
        
        self.max_trucks = [i for i in range(df_boxes.shape[0])]
        
        self.min_height = min_height
        self.min_width = min_breadth
        self.min_length = min_length
        
#         self.volume_of_truck = volume_of_truck
#         self.weight_of_truck = weight_of_truck
        
        self.x_axis = x_axis
        self.y_axis = y_axis
        self.z_axis = z_axis
        self.length_of_truck = length_of_truck
        self.breadth_of_truck = breadth_of_truck
        self.height_of_truck = height_of_truck
        self.weight_of_truck = weight_of_truck
        self.volume_of_truck = volume_of_truck
        self.max_weight_for_each_stack = 350
        
        return self

    def decision_variables(self, df_truck, df_boxes):
        #Decision Variable -> y (if truck i is used or not)
        y = [LpVariable("t{0}".format(i+1), cat=LpBinary) for i in self.max_trucks]

        #Decision Variablee -> boxes_loaded (1 if box i is loaded in truck j at position k)
        list_of_axes = [[x for x in range(self.x_axis)] , [y for y in range(self.y_axis)] , [z for z in range(self.z_axis)]]
        list_coordinates = list(itertools.product(*list_of_axes))
        box_loaded = LpVariable.dicts("At_truck_location",
                                           [(i,j, k) for i in df_boxes['boxes']
                                            for j in self.max_trucks
                                            for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]],
                                           0, 1, LpBinary)
        list_coordinates = list_coordinates
        return y, box_loaded, list_coordinates
    
    def build_constraints(self, df_boxes, problem, y, box_loaded, list_coordinates):
        #For each box, it can be placed in only one truck
        for i in df_boxes['boxes']:
            problem += lpSum(box_loaded[(i, j, k)] for j in self.max_trucks
                         for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) == 1

        #for each truck,
        #    summation of weight of all boxes should be less than total weight truck can carry
        #    summation of volume of all boxes should be less than volume of the truck
        for j in self.max_trucks:
            problem += lpSum(box_loaded[(i, j, k)] * df_boxes['weight'][i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= self.weight_of_truck * y[j]
            problem += lpSum(box_loaded[(i, j, k)] * df_boxes['volume'][i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= self.volume_of_truck * y[j]

        # for each truck,
        #     for each x,y point
        #         1. sum of all boxes placed along the z axis on that point should be less than height of truck
        #         2. weight of all boxes along a stack must be less than permissible weight
        #         3. surface area of a box must be less than or equal to that of the box placed below it
        for j in self.max_trucks:
            for y_ in range(self.y_axis):
                for x_ in range(self.x_axis):
                    problem += lpSum(box_loaded[(i, j, k)] * df_boxes['height'].iloc[i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(x_, y_, z_) for z_ in range(self.z_axis)]) <= self.height_of_truck * y[j]

                    problem += lpSum(box_loaded[(i, j, k)] * df_boxes['weight'].iloc[i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(x_, y_, z_) for z_ in range(self.z_axis)]) <= self.max_weight_for_each_stack * y[j]

        # for each truck,
        #     for each x,z point
        #         sum of all boxes placed along the y axis on that point should be less than width of truck
        for j in self.max_trucks:
            for z_ in range(self.z_axis):
                for x_ in range(self.x_axis):
        #             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
                    problem += lpSum(box_loaded[(i, j, k)] * df_boxes['breadth'].iloc[i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(x_, y_, z_) for y_ in range(self.y_axis)]) <= self.breadth_of_truck * y[j]

        # for each truck,
        #     for each y,z point
        #         sum of all boxes placed along the x axis on that point should be less than length of truck
        for j in self.max_trucks:
            for z_ in range(self.z_axis):
                for y_ in range(self.y_axis):
        #             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
                    problem += lpSum(box_loaded[(i, j, k)] * df_boxes['length'].iloc[i] for i in df_boxes['boxes'] 
                            for k in [pymorton.interleave(x_, y_, z_) for x_ in range(self.x_axis)]) <= self.length_of_truck * y[j]
            
        return problem
        
        
        
    def optimize_loading(self, df_truck, df_boxes):
        problem = LpProblem("truck_loading", LpMinimize)
        self.define_parameters(df_truck, df_boxes)
        y, box_loaded, list_coordinates = self.decision_variables(df_truck, df_boxes)
        problem += lpSum([y[i] for i in self.max_trucks])
        problem = self.build_constraints(df_boxes, problem, y, box_loaded, list_coordinates)
#         problem = self.problem
        problem.solve()
        
        TOL = 0.00001
        truck_num = 0
        total_trucks_used = 0
        for j in self.max_trucks:
            if y[j].varValue > TOL:
#                 print(f"val of truck_{j} is {y[j].varValue}")
                total_trucks_used += 1
        print(f"Total Trucks Used: {total_trucks_used}")
        print("\n")
        print("="* 100)
        for j in self.max_trucks:

            if y[j].varValue > TOL:
                print(f"Truck Number: {truck_num+1}")
                print("=" * 100)
                total_weight_loaded = 0
                total_num_boxes = 0
                total_volume_loaded = 0
                for i in df_boxes['boxes']:
                    for k in list_coordinates:
                        x_ = k[0]
                        y_ = k[1]
                        z_ = k[2]
                        if box_loaded[(i, j, pymorton.interleave3(x_, y_, z_))].varValue > TOL:
                            total_num_boxes += 1
                            total_weight_loaded += df_boxes['weight'][i]
                            total_volume_loaded += df_boxes['volume'][i]
                print(f'Total Boxes Loaded: {total_num_boxes}')
                print(f"Total Weight Loaded: {total_weight_loaded}")
                print(f"Total Volume Loaded: {total_volume_loaded}")
                print("-" * 80)

#                 for i in data['boxes']:
#                     if boxes_loaded[(i, j)].varValue > TOL:

#                         box_num = data['boxes'][i]
#                         len_ = data['length'][i]
#                         br_ = data['breadth'][i]
#                         hei_ = data['height'][i]
#                         wt_ = data['weight'][i]
#                         print(f"\t Box Number: {box_num}", f"\t Dimensions of Box: {len_, br_, hei_}", f"\t Weight of Box: {wt_}", sep = "\n")
#                         print("-" * 80)
        #         print ("Truck ", j, " holds ", \
        #         [i for i in data['boxes']
        #         if boxes_loaded[(i, j)].varValue > TOL])
                print("=" * 100)
                truck_num+= 1

In [ ]:
prob = OptimiseBoxLoading()

#### 1. Normal Boxes
Should fit in 1 truck!

In [ ]:
#Boxes Data

data = pd.DataFrame()
data['boxes'] = [0, 1]
data['weight'] =[150, 150]


data['length'] = [200, 200]
data['breadth'] = [150, 150]
data['height'] = [60, 60]
data['volume'] = list(np.array(data['length']) * np.array(data['breadth']) * np.array(data['height']))
#     df['truck_weight'] = 1000
data.head()


In [ ]:
truck_dict =dict({'truck_length': [200],
            'truck_width': [150],
            'truck_height': [120],
            'truck_weight_limit': [800],
            'truck_capacity': [200 * 150 * 120],
            'truck_name': 'truck'})
truck_df = pd.DataFrame.from_dict(truck_dict, orient = 'columns')
truck_df.head()

In [ ]:
prob.optimize_loading(truck_df, data)

#### 2. Boxes which exceed total weight in a stack constraint
Should require 2 trucks!

In [ ]:
#Boxes Data

data = pd.DataFrame()
data['boxes'] = [0, 1]
data['weight'] =[201, 150]


data['length'] = [200, 200]
data['breadth'] = [150, 150]
data['height'] = [60, 55]
data['volume'] = list(np.array(data['length']) * np.array(data['breadth']) * np.array(data['height']))
#     df['truck_weight'] = 1000
data.head()

In [ ]:
prob.optimize_loading(truck_df, data)

#### 3. Boxes which exceed the truck's length constraint
Should require 2 trucks!

In [ ]:
#Boxes Data

data = pd.DataFrame()
data['boxes'] = [0, 1]
data['weight'] =[100, 150]


data['length'] = [100, 110]
data['breadth'] = [150, 150]
data['height'] = [120, 120]
data['volume'] = list(np.array(data['length']) * np.array(data['breadth']) * np.array(data['height']))
#     df['truck_weight'] = 1000
data.head()

In [ ]:
prob.optimize_loading(truck_df, data)

#### Testing For an arbitrary number of boxes

In [ ]:
df = pd.DataFrame()
df['weight'] =[100] * 15 + [120]* 10
df['boxes'] = [i for i in range(len(df['weight']))]

df['length'] = [150] * 15 + [100] * 10
df['breadth'] = [100] * 15 + [80] * 10
df['height'] = [80] * 15 + [80] * 10
df['volume'] = list(np.array(df['length']) * np.array(df['breadth']) * np.array(df['height']))

In [ ]:
df.head(2)

In [ ]:
truck_dict =dict({'truck_length': [400],
            'truck_width': [280],
            'truck_height': [200],
            'truck_weight_limit': [1500],
            'truck_capacity': [400 * 280 * 200],
            'truck_name': 'truck'})
truck_df = pd.DataFrame.from_dict(truck_dict, orient = 'columns')
truck_df.head()

In [ ]:
import datetime

In [ ]:
time1 = datetime.datetime.now()

In [ ]:
prob.optimize_loading(truck_df, df)

In [ ]:
time2 = datetime.datetime.now()

In [ ]:
print(time2 - time1)

In [ ]:
problem